#2024 WASDE Release Dates (12:00pm ET)
#Aug. 12, Sep. 12, Oct. 11, Nov. 8, Dec. 10

#add data for 2024! 
#add global data! 
#think about how to avoid duplications 
#when new data comes insert only for 2022,2023, and 2024

In [1]:
import asyncio
import aiohttp
import pandas as pd
from requests.structures import CaseInsensitiveDict
from itertools import product
import nest_asyncio
from constants import years, commodity_codes

nest_asyncio.apply()  # This line allows asyncio.run() to be called in Jupyter or similar environments


successful_results = []
failed_results = []

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["API_KEY"] = "68F6B0D6-444E-43C0-A4E1-64CF5D3710B7"

DELAY_BETWEEN_BATCHES = 0.5  # Define the delay

async def fetch_data(session, year, commodity_code):
    try:
        url = f"https://apps.fas.usda.gov/PSDOnlineDataServices/api/CommodityData/GetWorldCommodityDataByYear?commodityCode={commodity_code}&marketYear={year}"
        async with session.get(url, headers=headers) as resp:
            # print(f"Year {year}, Commodity {commodity_code}: {resp.status}")
            if resp.status == 200:
                data = await resp.json()
                df = pd.DataFrame(data)
                # print(f"Data for Year {year}, Commodity {commodity_code}:")
                # print(df)
                return df, None
            else:
                return None, f"Failed to fetch data for year {year}, commodity {commodity_code} with status {resp.status}"
    except Exception as e:
        return None, str(e)

async def main():
    global successful_results, failed_results
    async with aiohttp.ClientSession() as session:
        tasks = []
        for year, commodity_code in product(years, commodity_codes):
            task = asyncio.ensure_future(fetch_data(session, year, commodity_code))
            tasks.append(task)
            await asyncio.sleep(DELAY_BETWEEN_BATCHES)
        
        results = await asyncio.gather(*tasks)

    successful_results = [result for result, error in results if result is not None]
    failed_results = [error for result, error in results if error is not None]

# Run the main function
await main()






In [2]:
b_ssh_host = '18.195.40.197'
b_ssh_user = 'forge'
b_ssh_port = 22
b_ssh_private_key = '/Users/germankosenkov/.ssh/id_rsa'
b_sql_hostname = 'gehrman.ccq0tjftm3pw.eu-central-1.rds.amazonaws.com'
b_sql_username = 'gehrman'
b_sql_password = 'ycEqAKjqFZH3sH'
b_sql_database = 'vesper'
b_sql_port = 3306

In [3]:
def query_data(ssh_host, ssh_user, ssh_port, ssh_private_key, sql_hostname, sql_username, sql_password, sql_database, sql_port, query):
    with SSHTunnelForwarder(
            (ssh_host, ssh_port),
            ssh_username=ssh_user,
            ssh_pkey=ssh_private_key,
            remote_bind_address=(sql_hostname, sql_port)) as tunnel:
        conn = pymysql.connect(
            host='127.0.0.1',
            user=sql_username,
            passwd=sql_password,
            db=sql_database,
            port=tunnel.local_bind_port
        )
        data = pd.read_sql_query(query, conn)
        conn.close()
    return data

In [8]:
from datetime import datetime, timedelta
from sshtunnel import SSHTunnelForwarder
from pymysql import IntegrityError, OperationalError
import pandas as pd
import pymysql
from dateutil import relativedelta
import json
import traceback
import logging
from constants import mapping


logger = logging.getLogger(__name__)
logger.setLevel(logging.CRITICAL)  # Set to CRITICAL during the insertion process to minimize overhead

country_id = 264

try:
    with SSHTunnelForwarder(
            (b_ssh_host, b_ssh_port),
            ssh_username=b_ssh_user,
            ssh_pkey=b_ssh_private_key,
            remote_bind_address=(b_sql_hostname, b_sql_port)) as tunnel:
        
        b_conn = pymysql.connect(
            host='127.0.0.1',
            user=b_sql_username,
            passwd=b_sql_password,
            db=b_sql_database,
            port=tunnel.local_bind_port
        )
        
        b_cursor = b_conn.cursor()

        try:
            # query = "SELECT id, name FROM countries"
            # country_df = pd.read_sql(query, b_conn)
            # country_df['name'] = country_df['name'].str.strip()
            # country_mapping = dict(zip(country_df['name'], country_df['id']))
            source_id = 4

            aggregated_results = []

            for df in successful_results:
                # Map commodity codes
                df['CommodityCode'] = df['CommodityCode'].str.strip()
                df['CommodityCode'] = df['CommodityCode'].map(mapping)
                
                # # Normalize and map country names
                # df['CountryName'] = df['CountryName'].str.strip().replace(country_name_exceptions)
                # df['CountryId'] = df['CountryName'].map(country_mapping).astype(pd.Int64Dtype())
                
                # Normalize and convert values
                df['Value'] = df['Value'].astype(float)
                mask = df['UnitDescription'].str.strip() == '(1000 MT)'
                df.loc[mask, 'Value'] *= 1000
                df['MarketYear'] = df['MarketYear'].astype(int)
                df['Value'] = df['Value'].round(2)
                df.loc[mask, 'UnitDescription'] = 'MT'

                # # Check for unmapped countries
                # unmapped_countries = df[df['CountryId'].isna()]['CountryName'].unique()
                # if len(unmapped_countries) > 0:
                #     print("Unmapped Countries:", unmapped_countries)
                
                aggregated_results.append(df)

            # Concatenate results
            if aggregated_results:
                final_result = pd.concat(aggregated_results, ignore_index=True)
                # print(final_result)
            else:
                print("No data to aggregate.")

            # Batch insert using executemany
            sql = "INSERT INTO annual_data_PSD (product_id, country_code, market_year, attribute_id, unit_id, amount, source_id) VALUES (%s, %s, %s, %s, %s, %s, %s)"
            values = [
                (
                    row["CommodityCode"], country_id, row["MarketYear"], row["AttributeId"], row["UnitId"], row["Value"], source_id
                )
                for index, row in final_result.iterrows()
            ]
            b_cursor.executemany(sql, values)
            b_conn.commit()  
            b_conn.close()

        except IntegrityError as ie:
            logger.error(f"Integrity error occurred: {ie}")
            b_conn.rollback()
            logger.info("Transaction rolled back due to IntegrityError.")

        except Exception as e:
            logger.error(f"An unexpected error occurred during query execution: {e}")
            b_conn.rollback()
            logger.info("Transaction rolled back due to an unexpected error.")

        finally:
            b_cursor.close()
            logger.info("Cursor closed.")

except Exception as e:
    logger.critical(f"Critical error in establishing SSH Tunnel: {e}")


In [ ]:
#2024 WASDE Release Dates (12:00pm ET)
#Aug. 12, Sep. 12, Oct. 11, Nov. 8, Dec. 10